In [12]:
import numpy as np
import pandas as pd

In [26]:
#songs = pd.read_csv('data/songs.csv')
#user = pd.read_csv('data/user.csv')
spot = pd.read_csv('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/train.csv')[['artists','album_name','song name', 'track_genre', 'danceability', 'energy', 'loudness', 'tempo', 'liveness', 'valence' ]]

In [27]:
print(spot.head())


                  artists                                         album_name  \
0             Gen Hoshino                                             Comedy   
1            Ben Woodward                                   Ghost (Acoustic)   
2  Ingrid Michaelson;ZAYN                                     To Begin Again   
3            Kina Grannis  Crazy Rich Asians (Original Motion Picture Sou...   
4        Chord Overstreet                                            Hold On   

                    song name track_genre  danceability  energy  loudness  \
0                      Comedy    acoustic         0.676  0.4610    -6.746   
1            Ghost - Acoustic    acoustic         0.420  0.1660   -17.235   
2              To Begin Again    acoustic         0.438  0.3590    -9.734   
3  Can't Help Falling In Love    acoustic         0.266  0.0596   -18.515   
4                     Hold On    acoustic         0.618  0.4430    -9.681   

     tempo  liveness  valence  
0   87.917    0.3580    

In [1]:
from h2ogpte import H2OGPTE

In [32]:
client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-ms7SU43E34tS9UJks5RD2KM3m1JumOR2pM73Dk95VzKjM6TZ',
)

# # Chat with LLM
# chat_session_id = client.create_chat_session()
# with client.connect(chat_session_id) as session:
#     # Simple Question for Document Collection
#     answer = session.query(
#         "Can you hallucinate?",
#     ).content
#     print(answer)

# Create Collection
collection_id = client.create_collection(
    name="Music Recommendation",
    description="PDF -> text -> summary",
)

import os

# Your existing code
file_path = "/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/user.xlsx"
with open(file_path, "rb") as f:
    user = client.upload(os.path.basename(file_path), f)

with open('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/listening history.xlsx', 'rb') as f:
    hist = client.upload('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/listening history.xlsx', f)

with open('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/songs_short.xlsx', 'rb') as f:
    songs = client.upload('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/songs_short.xlsx', f)

with open('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/train.csv', 'rb') as f:
    spot = client.upload('/Users/blabbyduck/Desktop/Y3S2/DSA4213/LLMasters/data/train.csv', f)

# # Converting the input into chunked text and embeddings...
client.ingest_uploads(collection_id, [hist, user, spot])

KeyboardInterrupt: 

In [34]:
chat_session_id = client.create_chat_session(collection_id)
with client.connect(chat_session_id) as session:
    # Simple Question for Document Collection
        answer = session.query(
        message="Can you recommend similar song to FRIENDS",
        system_prompt = 'Assume music and song to be the same word. When a question as for a similar music, recommend less than 5 music unless told otherwise. Find similar music based on genre and other factors such as danceability, loudness, speechiness and more' ,
        rag_config={
            "rag_type": "rag",
        },
        ).content
        print(answer)


Prompt designs

In [9]:
print([x["base_model"] for x in client.get_llms()])

['mistralai/Mixtral-8x7B-Instruct-v0.1', 'h2oai/h2ogpt-4096-llama2-70b-chat', 'h2oai/h2ogpt-4096-llama2-13b-chat', 'h2oai/h2ogpt-32k-codellama-34b-instruct', 'HuggingFaceH4/zephyr-7b-beta', 'NousResearch/Nous-Capybara-34B', 'claude-2.1', 'gpt-3.5-turbo-0613', 'gpt-3.5-turbo-16k-0613', 'gpt-35-turbo-1106', 'gpt-4-1106-preview', 'gemini-pro', 'mistral-small-latest', 'mistral-large-latest', 'mistral-medium']


In [10]:
# Inspect and collect all text chunks
chunks = []
for chunk_id in range(1, 100):
    try:
        chunk = client.get_chunks(collection_id, [chunk_id])
        print(chunk, flush=True)
        chunks.append(chunk[0].text)
    except:
        break

print(f"Number of chunks: {len(chunks)}", flush=True)

[Chunk(text='ID\nPassword Name\nGenre 1\nGenre 2\nGenre 3\n1\n1 Sibyl Bode\nrock\nr&b\npop\n2\n2 Eleazar Hettinger\nrap\nsoul\nhip hop\n3\n3 Geovanny Lang\nrap\ncountry\nsoul\n4\n4 Micah Satterfield\njazz\nr&b\nsoul\n5\n5 Courtney Schowalter soul\nfolk\nmetal\n6\n6 Donato Daniel\nmetal\nr&b\npop\n7\n7 Caroline Jaskolski\nr&b\nk-pop\nelectronic\n8\n8 Vincenza Collins\npop\nfolk\nelectronic\n9\n9 Judd Becker\nk-pop\nfolk\nr&b\n10\n10 Nathanial Walsh\nsoul\nmetal\nrap\n11\n11 Korbin Kirlin\nrap\ncountry\nk-pop\n12\n12 Kaleigh McGlynn\nhip hop\njazz\nr&b\n13\n13 Marielle Wolf\nmetal\npop\nsoul\n14\n14 Marcos Raynor\nrock\nelectronic\nsoul\n15\n15 Eddie Ebert\nfolk\nsoul\nrap\n16\n16 Jaiden Jast\nr&b\nrock\nrap\n17\n17 Abigale Kub\nfolk\nsoul\ncountry\n18\n18 Ana Williamson\nk-pop\nfolk\nhip hop\n19\n19 Lora Berge\nr&b\ncountry\nrock\n20\n20 Mafalda MacGyver\nsoul\nrap\nrock\n21\n21 Nolan Kihn\nhip hop\nrap\ncountry\n22\n22 Camila Stoltenberg\nfolk\njazz\nsoul\n23\n23 Josie Wisoky\nelectron

In [ ]:
Prompt_template = """
Answer the questions based on only the following context:

{context}

---

Answer the questions based on the above context: {query}
"""
